# 1.2 Document Formats & Metadata - Ungraph

Este notebook compara los diferentes formatos de documentos soportados y cómo se extraen sus metadatos.

## Objetivos

1. **Comparar formatos** - Markdown, TXT, Word, PDF lado a lado
2. **Extracción de metadatos** - Qué metadatos se extraen de cada formato
3. **Manejo de encoding** - Detección automática en archivos de texto
4. **Mejores prácticas** - Cuándo usar cada formato

**Referencias:**
- [Guía de Ingesta](../../docs/guides/ingestion.md)


In [1]:
def add_src_to_path(path_folder: str):
    import sys
    from pathlib import Path
    base_path = Path().resolve()
    for parent in [base_path] + list(base_path.parents):
        candidate = parent / path_folder
        if candidate.exists():
            parent_dir = candidate.parent
            if str(parent_dir) not in sys.path:
                sys.path.insert(0, str(parent_dir))
            if str(candidate) not in sys.path:
                sys.path.append(str(candidate))
            return

add_src_to_path(path_folder="src")
add_src_to_path(path_folder="src/utils")
add_src_to_path(path_folder="src/data")

try:
    import ungraph
except ImportError:
    import src
    ungraph = src

from infrastructure.services.langchain_document_loader_service import LangChainDocumentLoaderService
from src.utils.handlers import find_in_project

print(f"📦 Ungraph version: {ungraph.__version__}")


📦 Ungraph version: 0.1.0


## Parte 1: Comparar Carga de Formatos

Carguemos el mismo contenido conceptual en diferentes formatos y comparemos los resultados.


In [2]:
# Encontrar archivos de ejemplo
data_path = find_in_project("data", "folder", None)

# Crear servicio de carga
loader_service = LangChainDocumentLoaderService()

# Cargar diferentes formatos
formats_data = {}

# Markdown
md_file = data_path / "110225.md"
if md_file.exists():
    docs_md = loader_service.load(md_file, clean=False)
    formats_data["Markdown"] = {"file": md_file, "docs": docs_md}

# Texto
txt_file = data_path / "AnnyLetter.txt"
if txt_file.exists():
    docs_txt = loader_service.load(txt_file, clean=False)
    formats_data["Texto"] = {"file": txt_file, "docs": docs_txt}

# Word
docx_file = data_path / "Usar símboles de silencio de corchea.docx"
if docx_file.exists():
    docs_docx = loader_service.load(docx_file, clean=False)
    formats_data["Word"] = {"file": docx_file, "docs": docs_docx}

# PDF
pdf_files = list(data_path.glob("*.pdf"))
if pdf_files:
    pdf_file = pdf_files[0]
    try:
        docs_pdf = loader_service.load(pdf_file, clean=False)
        formats_data["PDF"] = {"file": pdf_file, "docs": docs_pdf}
    except Exception as e:
        print(f"⚠️  Error cargando PDF: {e}")

print(f"✅ Formatos cargados: {list(formats_data.keys())}")


2025-12-25 21:11:59,533 - INFO - Encontrado: D:\projects\Ungraph\src\data
2025-12-25 21:11:59,536 - INFO - Cargando archivo Markdown: D:\projects\Ungraph\src\data\110225.md


2025-12-25 21:12:02,321 - INFO - Archivo cargado exitosamente. Documentos generados: 1
2025-12-25 21:12:02,322 - INFO - Cargando archivo de texto: D:\projects\Ungraph\src\data\AnnyLetter.txt
2025-12-25 21:12:02,361 - INFO - Codificación detectada por chardet: iso-8859-1 (confianza: 73.00%)
2025-12-25 21:12:02,364 - INFO - Codificación detectada: iso-8859-1
2025-12-25 21:12:02,367 - INFO - Archivo cargado exitosamente con codificación: iso-8859-1
2025-12-25 21:12:02,368 - INFO - Archivo cargado exitosamente. Documentos generados: 1
2025-12-25 21:12:02,371 - INFO - Cargando archivo Word: D:\projects\Ungraph\src\data\Usar símboles de silencio de corchea.docx
2025-12-25 21:12:02,812 - INFO - Archivo cargado exitosamente. Documentos generados: 1
2025-12-25 21:12:02,816 - INFO - Cargando archivo PDF con Docling: D:\projects\Ungraph\src\data\peach-et-al-2014-qsar-modeling-of-imbalanced-high-throughput-screening-data-in-pubchem.pdf
2025-12-25 21:12:04,169 - INFO - detected formats: [<InputForm

✅ Formatos cargados: ['Markdown', 'Texto', 'Word', 'PDF']


## Parte 2: Comparar Metadatos Extraídos

Comparemos qué metadatos se extraen de cada formato.


In [3]:
# Comparar metadatos por formato
print("📊 Comparación de Metadatos por Formato\n")
print("=" * 80)

for format_name, data in formats_data.items():
    print(f"\n{format_name} ({data['file'].name}):")
    print("-" * 80)
    
    if data['docs']:
        doc = data['docs'][0]
        print(f"  Contenido: {len(doc.content)} caracteres")
        print(f"  Metadatos extraídos:")
        
        for key, value in doc.metadata.items():
            if isinstance(value, (str, int, float)):
                display_value = str(value)[:100] if len(str(value)) > 100 else str(value)
                print(f"    - {key}: {display_value}")
            else:
                print(f"    - {key}: {type(value).__name__}")
    else:
        print("  ⚠️  No se pudieron cargar documentos")


📊 Comparación de Metadatos por Formato


Markdown (110225.md):
--------------------------------------------------------------------------------
  Contenido: 6676 caracteres
  Metadatos extraídos:
    - file_path: D:\projects\Ungraph\src\data\110225.md
    - source: D:\projects\Ungraph\src\data\110225.md

Texto (AnnyLetter.txt):
--------------------------------------------------------------------------------
  Contenido: 17975 caracteres
  Metadatos extraídos:
    - file_path: D:\projects\Ungraph\src\data\AnnyLetter.txt
    - encoding: iso-8859-1
    - source: D:\projects\Ungraph\src\data\AnnyLetter.txt

Word (Usar símboles de silencio de corchea.docx):
--------------------------------------------------------------------------------
  Contenido: 22030 caracteres
  Metadatos extraídos:
    - file_path: D:\projects\Ungraph\src\data\Usar símboles de silencio de corchea.docx
    - source: D:\projects\Ungraph\src\data\Usar símboles de silencio de corchea.docx

PDF (peach-et-al-2014-qsar-mode

## Parte 3: Metadatos Específicos por Formato

Cada formato tiene metadatos únicos que pueden ser útiles.


In [4]:
# Analizar metadatos específicos
print("📋 Metadatos Específicos por Formato\n")

# Markdown - puede tener información de estructura
if "Markdown" in formats_data and formats_data["Markdown"]["docs"]:
    md_doc = formats_data["Markdown"]["docs"][0]
    print("Markdown:")
    print(f"  - file_type: {md_doc.metadata.get('file_type', 'N/A')}")
    print(f"  - filename: {md_doc.metadata.get('filename', 'N/A')}")
    if 'source' in md_doc.metadata:
        print(f"  - source: {md_doc.metadata['source']}")

# PDF - puede tener page_number, document_structure, tables, images
if "PDF" in formats_data and formats_data["PDF"]["docs"]:
    pdf_doc = formats_data["PDF"]["docs"][0]
    print("\nPDF (metadatos avanzados de Docling):")
    for key in ['page_number', 'document_structure', 'tables', 'images']:
        if key in pdf_doc.metadata:
            print(f"  - {key}: {pdf_doc.metadata[key]}")

# Word - puede tener información de formato
if "Word" in formats_data and formats_data["Word"]["docs"]:
    word_doc = formats_data["Word"]["docs"][0]
    print("\nWord:")
    print(f"  - file_type: {word_doc.metadata.get('file_type', 'N/A')}")
    if 'source' in word_doc.metadata:
        print(f"  - source: {word_doc.metadata['source']}")


📋 Metadatos Específicos por Formato

Markdown:
  - file_type: N/A
  - filename: N/A
  - source: D:\projects\Ungraph\src\data\110225.md

PDF (metadatos avanzados de Docling):

Word:
  - file_type: N/A
  - source: D:\projects\Ungraph\src\data\Usar símboles de silencio de corchea.docx


## Parte 4: Manejo de Encoding en Archivos de Texto

Los archivos de texto pueden tener diferentes encodings. Ungraph los detecta automáticamente.


In [5]:
# Verificar encoding de archivos de texto
import chardet

txt_file = data_path / "AnnyLetter.txt"
if txt_file.exists():
    with open(txt_file, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        print(f"📄 Archivo: {txt_file.name}")
        print(f"   Encoding detectado: {result['encoding']}")
        print(f"   Confianza: {result['confidence']:.2%}")
        print(f"   Tamaño: {len(raw_data)} bytes")
        
        # Intentar leer con diferentes encodings
        print(f"\n   Pruebas de lectura:")
        for encoding in ['utf-8', 'windows-1252', 'latin-1']:
            try:
                content = raw_data.decode(encoding)
                print(f"     ✅ {encoding}: {len(content)} caracteres")
            except:
                print(f"     ❌ {encoding}: Error")


📄 Archivo: AnnyLetter.txt
   Encoding detectado: ISO-8859-1
   Confianza: 73.00%
   Tamaño: 18219 bytes

   Pruebas de lectura:
     ❌ utf-8: Error
     ✅ windows-1252: 18219 caracteres
     ✅ latin-1: 18219 caracteres


## Resumen y Comparación

### Comparación de Formatos

| Formato | Ventajas | Limitaciones | Mejor Para |
|---------|----------|--------------|------------|
| **Markdown** | Estructura preservada, fácil de procesar | Requiere formato específico | Documentación, artículos estructurados |
| **Texto** | Universal, simple | Sin estructura | Texto plano, logs |
| **Word** | Formato común, estructura preservada | Puede tener formato complejo | Documentos de oficina |
| **PDF** | Formato estándar, estructura avanzada | Requiere librerías especiales | Documentos académicos, informes |

### Metadatos Comunes

Todos los formatos incluyen:
- `filename`: Nombre del archivo
- `file_type`: Tipo de archivo (md, txt, docx, pdf)
- `file_path`: Ruta completa al archivo

### Metadatos Específicos

- **PDF**: `page_number`, `document_structure`, `tables`, `images` (si Docling está disponible)
- **Markdown**: Información de estructura de headers
- **Word**: Información de formato y estilo

### Mejores Prácticas

1. **Elegir formato según necesidad**: PDF para documentos complejos, Markdown para estructura simple
2. **Verificar metadatos**: Revisa qué metadatos se extraen para tu caso de uso
3. **Encoding**: Los archivos de texto se detectan automáticamente, pero verifica si hay problemas
4. **PDF avanzado**: Usa Docling para mejor extracción de estructura y tablas

## Referencias

- [Guía de Ingesta](../../docs/guides/ingestion.md)
- [Document Loader Service](../../src/infrastructure/services/langchain_document_loader_service.py)
